In [1]:
import numpy
from keras.models import load_model

In [2]:
seg_max_min=[(315.0, 68.0),
 (78.0, 29.0),
 (308.0, 34.0),
 (187.0, 34.0),
 (68.0, 22.0),
 (74.8, 37.4),
 (77.0, 34.0),
 (517.2, 63.0),
 (381.8, 68.0),
 (50.0, 24.0),
 (287.4, 50.6),
 (237.0, 46.0),
 (142.0, 58.0),
 (202.0, 61.0),
 (494.0, 42.0),
 (144.0, 65.6),
 (208.4, 51.0),
 (187.0, 63.0),
 (149.4, 43.0),
 (351.0, 68.0)]

In [45]:
train=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\train.npy',allow_pickle=True).item()
test=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\test.npy',allow_pickle=True).item()
dw_train_0=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\train_day_of_week.npy')
dw_test_0=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\test_day_of_week.npy')
ts=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\encoded_time_stamp.npy')

In [73]:
dw_train_0=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\train_day_of_week.npy')
dw_test_0=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\test_day_of_week.npy')

In [86]:
train_in_c=[]
train_in_d=[]
train_in_w=[]
train_out_c=[]
test_in_c=[]
test_in_d=[]
test_in_w=[]
test_out_c=[]
target_seg=[9, 24, 42, 83, 256, 267, 268, 277]
target_seg_len=len(target_seg)
for seg in target_seg:#路段key
    train_in_c+=( train[seg][0])
    train_in_d+=( train[seg][1])
    train_in_w+=( train[seg][2])
    train_out_c+=( train[seg][3])
    test_in_c+=(test[seg][0])
    test_in_d+=(test[seg][1])
    test_in_w+=(test[seg][2])
    test_out_c+=(test[seg][3])
dw_train=numpy.repeat(dw_train_0,target_seg_len)#路段数
dw_test=numpy.repeat(dw_test_0,target_seg_len)
ts_train=[]
for i in range(33):
    ts_train=numpy.concatenate((ts_train,ts[:176]))
ts_test=[]
for i in range(8):
    ts_test=numpy.concatenate((ts_test,ts[:176]))
ts_train=numpy.repeat(ts_train,target_seg_len)#路段数目
ts_test=numpy.repeat(ts_test,target_seg_len)

In [87]:
train_in_c=numpy.array(train_in_c)
test_in_c=numpy.array(test_in_c)
train_in_d=numpy.array(train_in_d)
test_in_d=numpy.array(test_in_d)
train_in_w=numpy.array(train_in_w)
test_in_w=numpy.array(test_in_w)
train_out_c=numpy.array(train_out_c)
test_out_c=numpy.array(test_out_c)
train_in_c=train_in_c.reshape(train_in_c.shape[0],4,1)
train_out_c=train_out_c.reshape(train_out_c.shape[0],1)
test_in_c=test_in_c.reshape(test_in_c.shape[0],4,1)
test_out_c=test_out_c.reshape(test_out_c.shape[0],1)
train_in_d=train_in_d.reshape(train_in_d.shape[0],4,1)
test_in_d=test_in_d.reshape(test_in_d.shape[0],4,1)
train_in_w=train_in_w.reshape(train_in_w.shape[0],1,1)
test_in_w=test_in_w.reshape(test_in_w.shape[0],1,1)
ts_train=ts_train.reshape(ts_train.shape[0],1,1)
ts_test=ts_test.reshape(ts_test.shape[0],1,1)
dw_test=dw_test.reshape(dw_test.shape[0],1,1)
dw_train=dw_train.reshape(dw_train.shape[0],1,1)
ts_train=ts_train.reshape(ts_train.shape[0],1,1)
ts_test=ts_test.reshape(ts_test.shape[0],1,1)
dw_test=dw_test.reshape(dw_test.shape[0],1,1)
dw_train=dw_train.reshape(dw_train.shape[0],1,1)
model_in=[train_in_c,train_in_d,train_in_w,dw_train,ts_train]
model_out=[train_out_c]
test_model_in=[test_in_c,test_in_d,test_in_w,dw_test,ts_test]
test_model_out=[test_out_c]

In [70]:
c0=load_model(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\log_0_c0.h5')

In [71]:
#cluster中元素对应的index
valid_index=[1, 4, 5]
max_arr=numpy.array([ [seg_max_min[i][0]] for i in valid_index])
min_arr=numpy.array([[seg_max_min[i][1]] for i in valid_index])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,8*176,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,8*176,axis=0)

In [76]:
pred=c0.predict(test_model_in)
pred=pred*bias_arr+min_arr
gt=(numpy.array(test_model_out[0])*bias_arr+min_arr)
tmp=numpy.absolute(pred-gt)/gt
numpy.mean(tmp)

0.06749629335185091

In [77]:
sum_mape=numpy.concatenate((sum_mape,tmp),axis=0)

In [78]:
c1=load_model(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\log_0_c1.h5')

In [81]:
#cluster中元素对应的index
valid_index=[0, 3, 7]
max_arr=numpy.array([ [seg_max_min[i][0]] for i in valid_index])
min_arr=numpy.array([[seg_max_min[i][1]] for i in valid_index])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,8*176,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,8*176,axis=0)

In [82]:
pred=c1.predict(test_model_in)
pred=pred*bias_arr+min_arr
gt=(numpy.array(test_model_out[0])*bias_arr+min_arr)
tmp=numpy.absolute(pred-gt)/gt
numpy.mean(tmp)

0.16192235641884953

In [83]:
sum_mape=numpy.concatenate((sum_mape,tmp),axis=0)

In [64]:
c2=load_model(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\log_0_c2.h5')

In [65]:
#cluster中元素对应的index
valid_index=[2, 6]
max_arr=numpy.array([ [seg_max_min[i][0]] for i in valid_index])
min_arr=numpy.array([[seg_max_min[i][1]] for i in valid_index])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,8*176,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,8*176,axis=0)

In [66]:
pred=c2.predict(test_model_in)
pred=pred*bias_arr+min_arr
gt=(numpy.array(test_model_out[0])*bias_arr+min_arr)
numpy.mean(numpy.absolute(pred-gt)/gt)

0.10426998911400301

In [67]:
sum_mape=numpy.absolute(pred-gt)/gt

In [84]:
sum_mape.shape

(11264, 1)

In [85]:
numpy.mean(sum_mape)

0.1120994909425134

In [88]:
c_all=load_model(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_0\log_0_0.h5')

In [89]:
#cluster中元素对应的index
valid_index=[i for i in range(8)]
max_arr=numpy.array([ [seg_max_min[i][0]] for i in valid_index])
min_arr=numpy.array([[seg_max_min[i][1]] for i in valid_index])
bias_arr=max_arr-min_arr
bias_arr=numpy.repeat(bias_arr,8*176,axis=0) #测试集天数*每天记录数
min_arr=numpy.repeat(min_arr,8*176,axis=0)

In [90]:
pred=c_all.predict(test_model_in)
pred=pred*bias_arr+min_arr
gt=(numpy.array(test_model_out[0])*bias_arr+min_arr)
numpy.mean(numpy.absolute(pred-gt)/gt)

0.10504914005703152

In [99]:
i=1
pred_c0=pred[i*176*8:(i+1)*176*8]
gt_c0=gt[i*176*8:(i+1)*176*8]
bias_arr_c0=bias_arr[i*176*8:(i+1)*176*8]
min_arr_c0=min_arr[i*176*8:(i+1)*176*8]
for i in [4, 5]:
    pred_c0=numpy.concatenate((pred_c0,pred[i*176*8:(i+1)*176*8]),axis=0) 
    gt_c0=numpy.concatenate((gt_c0,gt[i*176*8:(i+1)*176*8]),axis=0) 
    bias_arr_c0=numpy.concatenate((bias_arr_c0,bias_arr[i*176*8:(i+1)*176*8]),axis=0)
    min_arr_c0=numpy.concatenate((min_arr_c0,min_arr[i*176*8:(i+1)*176*8]),axis=0)
    # bias_arr_c0+=bias_arr[i*176*8:(i+1)*176*8]
    # min_arr_c0+=min_arr[i*176*8:(i+1)*176*8]

In [100]:

numpy.mean(numpy.absolute(pred_c0-gt_c0)/gt_c0)

0.06164752503283185